In [3]:
import random
import numpy as np
import math
import pandas as pd
from sklearn.metrics import mean_squared_error
import numdifftools as nd
import inspect
from matplotlib import pyplot as plt
import statistics

In [4]:
random.seed(10)

# (A)

Assumptions:
- X distributed uniformly from (0,1) INCLUSIVE

## getData(N, var):

In [5]:
def getData(N, var=1.0):
    
    if(N<1 or var<0):
        return "ERROR"
    
    
    xes = []
    yes = []
    zes = []
    
    for i in range(N):
        x = random.uniform(0, 1)
        z = np.random.normal(loc=0.0, scale=math.sqrt(var))
        y = math.cos(2*math.pi*x)+z
        
        xes.append(x)
        yes.append(y)
        #zes.append(z)
        
    #df = pd.DataFrame(list(zip(xes,yes,zes)), columns =['X', 'Y','Z'])
    df = pd.DataFrame(list(zip(xes,yes)), columns =['X', 'Y'])
    
    return df
        

In [6]:
# CHECK

test_df = getData(10,2.0)

test_df

,X,Y
0,0.571403,-1.982518
1,0.428889,-1.816969
2,0.578091,0.825640
3,0.206098,-1.421772
4,0.813321,0.251154
5,0.823589,-2.326334
6,0.653473,-3.295630
7,0.160230,0.939546
8,0.520669,-1.051852
9,0.327773,1.506792


# (B)

## Helper Functions

In [7]:
def getPolynomialY(poly_coefs, x):
    
    y = 0
    
    for i in range(len(poly_coefs)):
        y += poly_coefs[i] * (x**i)
        
    return y

In [8]:
def getPrediction(coefs, xes):
    
    y_preds = []
    
    for x in xes:
        y_preds.append(getPolynomialY(coefs,x))
        
    return y_preds

In [9]:
# CHECK

# y = 2 + 3x + (-5)x^2 + 0x^3 + 6x^4
# Online Calculator: (2,84) , (-0.1, 1.6506)

print("x = 2, y =(true) 84 =(pred) ", getPolynomialY([2, 3, -5, 0, 6], 2))
print("x = (-0.1), y =(true) 1.6506 =(pred) ", getPolynomialY([2, 3, -5, 0, 6], -0.1))

x = 2, y =(true) 84 =(pred)  84
x = (-0.1), y =(true) 1.6506 =(pred)  1.6505999999999998


## getMSE(df, poly_coefs):

In [10]:
def getMSE(df, poly_coefs):
    
    N = len(df['X'])
    
    y_preds = getPrediction(poly_coefs, df['X'])
        
    y_residual = df['Y'] - y_preds
    
    return np.sum(np.dot(y_residual.T , y_residual)) / len(df['Y'] - y_residual)

In [11]:
# CHECK

Y_pred = []

for i in range(len(test_df['X'])):
    Y_pred.append( getPolynomialY([1,2,3], test_df['X'][i]) )

print(getMSE(test_df,[1,2,3]), "(pred)=(true)", mean_squared_error(test_df['Y'],Y_pred))

18.93404880271201 (pred)=(true) 18.93404880271201


# (C)

## Helper Functions

In [12]:
def transpose(matrix):
    return np.array(matrix).T.tolist()

In [13]:
def augmentedXDataframe(df, d):
    
    returnDf = df.copy()
    
    for i in range(d+1):
        returnDf['X_'+str(i)] = [x**i for x in df['X']]
        
    return returnDf

In [14]:
#Check 

augmented_test_df = augmentedXDataframe(test_df, 3)   
augmented_test_df

,X,Y,X_0,X_1,X_2,X_3
0,0.571403,-1.982518,1.0,0.571403,0.326501,0.186563
1,0.428889,-1.816969,1.0,0.428889,0.183946,0.078892
2,0.578091,0.825640,1.0,0.578091,0.334190,0.193192
3,0.206098,-1.421772,1.0,0.206098,0.042476,0.008754
4,0.813321,0.251154,1.0,0.813321,0.661491,0.538005
5,0.823589,-2.326334,1.0,0.823589,0.678299,0.558639
6,0.653473,-3.295630,1.0,0.653473,0.427026,0.279050
7,0.160230,0.939546,1.0,0.160230,0.025674,0.004114
8,0.520669,-1.051852,1.0,0.520669,0.271097,0.141152
9,0.327773,1.506792,1.0,0.327773,0.107435,0.035214


In [102]:
def MiniBatchStochasticGradientDescent(df, d, learning_rate, epochs, details=False):
    
    augmented_df = augmentedXDataframe(df, d)      #Add polynomial X terms to dataframe
    
    N = len(augmented_df['X'])
    
    # START FROM A RANDOM NON-ZERO THETA _____________________
    
    theta = []
    
    for i in range(d+1):
        num = 0
        while num==0:
            num = random.randint(-10,10)
        theta.append(num)
        
    if details: print("Random Theta: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
    
    # UPDATE THETA EPOCHS TIMES ______________________________
    
    for k in range(epochs):
        
        if details: print("_______________________________________________________________")
            
        # Choose random indexes in N for batch
        randomIndexes = []
        batch_size= 1             #Stochastic gradient descent to save time
        
        for i in range(batch_size):  
            randomIndexes.append(random.randint(0, N-1))
        
        for j in range(d+1):  # for each coefficient
            
            derivative = 0
            
            # GET DERIVATIVE for theta[j]
            for i in randomIndexes:
                derivative += 2 * (df['Y'][i] - getPolynomialY(theta, df['X'][i])) * augmented_df['X_'+str(j)][i]
            
            # UPDATE THETA
            theta[j] += learning_rate * derivative
        
        if details: print("Epoch: "+str(k)+", Coefs: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
        
    return theta
    
    
    return estim_poly_coefs

#MiniBatchStochasticGradientDescent(test_df, 3, 0.01, 1500, True)

## fitData(df, d, var, gd_type, learning_rate, mini_batch_size, epochs, testing_size, details, weightDecay):

In [103]:
def fitData(df, d, var, weightDecay, learning_rate=0.01, mini_batch_size = 10, epochs = 1500, testing_size=2000, details=False):

    # (1)
    if weightDecay:
        estim_poly_coefs = MiniBatchStochasticGradientDescent_WeightDecay(df, d, learning_rate, epochs, details)
    else:
        estim_poly_coefs = MiniBatchStochasticGradientDescent(df, d, learning_rate, epochs, details)
    
    # (2)
    E_in = getMSE(df, estim_poly_coefs)
    
    #(3)
    E_out = getMSE(getData(testing_size, var), estim_poly_coefs)
        
    return (estim_poly_coefs, E_in, E_out)

In [104]:
# Check

fitData(df=test_df, d=2, weightDecay=False, learning_rate=0.01, testing_size=2000, epochs = 1500, var=2.0, details=True)

Random Theta:  [10, -7, -10] MSE: 29.91258
_______________________________________________________________
Epoch: 0, Coefs:  [9.860099939020031, -6.994899294857371, -9.97234342577885] MSE: 28.79595
_______________________________________________________________
Epoch: 1, Coefs:  [9.682244829604253, -7.094817390271859, -10.030728313276764] MSE: 27.05715
_______________________________________________________________
Epoch: 2, Coefs:  [9.594767643336526, -7.068277922042244, -9.994234676823778] MSE: 26.49035
_______________________________________________________________
Epoch: 3, Coefs:  [9.420745125479531, -7.147982483211167, -10.03616042429579] MSE: 24.99009
_______________________________________________________________
Epoch: 4, Coefs:  [9.259301834108399, -7.232616252297846, -10.08192183810036] MSE: 23.66624
_______________________________________________________________
Epoch: 5, Coefs:  [9.06066321101388, -7.321035459992952, -10.122908363513455] MSE: 22.2044
______________________

Epoch: 181, Coefs:  [4.857780299733474, -5.090667876344504, -7.056906572609438] MSE: 7.41337
_______________________________________________________________
Epoch: 182, Coefs:  [4.695301260293708, -5.136319966474965, -7.072027471991853] MSE: 7.14362
_______________________________________________________________
Epoch: 183, Coefs:  [4.688367738426853, -5.084033319159193, -7.0181814731348595] MSE: 7.10237
_______________________________________________________________
Epoch: 184, Coefs:  [4.813829340485912, -4.985703198785242, -6.940058479328518] MSE: 7.29993
_______________________________________________________________
Epoch: 185, Coefs:  [4.7483052755276765, -5.0232951334424145, -6.962478472622017] MSE: 7.17999
_______________________________________________________________
Epoch: 186, Coefs:  [4.625580857660553, -5.054965665068815, -6.9727579753156865] MSE: 6.98129
_______________________________________________________________
Epoch: 187, Coefs:  [4.618379817840618, -5.00348302373

_______________________________________________________________
Epoch: 328, Coefs:  [2.927616491834218, -3.748848484804346, -5.252809277269037] MSE: 4.15469
_______________________________________________________________
Epoch: 329, Coefs:  [2.9100114150481136, -3.750250045802854, -5.252604524780007] MSE: 4.14651
_______________________________________________________________
Epoch: 330, Coefs:  [3.024108109310678, -3.6692795612682967, -5.192624764580118] MSE: 4.17226
_______________________________________________________________
Epoch: 331, Coefs:  [3.1301932546667492, -3.59375079684291, -5.136520953986733] MSE: 4.25188
_______________________________________________________________
Epoch: 332, Coefs:  [3.218772062828006, -3.524404927750488, -5.081489435854854] MSE: 4.35927
_______________________________________________________________
Epoch: 333, Coefs:  [3.1371172551285493, -3.575629734879362, -5.114391669085841] MSE: 4.25467
_______________________________________________________

Epoch: 474, Coefs:  [1.9334464018477595, -2.7986712813528207, -4.065840658894426] MSE: 2.99399
_______________________________________________________________
Epoch: 475, Coefs:  [1.8998818549475753, -2.802973458831589, -4.0659569804646205] MSE: 2.98955
_______________________________________________________________
Epoch: 476, Coefs:  [1.985543061630705, -2.744604892394124, -4.021944635286162] MSE: 2.98205
_______________________________________________________________
Epoch: 477, Coefs:  [1.9808697666132833, -2.7513472809343207, -4.027137066984396] MSE: 2.98314
_______________________________________________________________
Epoch: 478, Coefs:  [1.9632016038074624, -2.741460211470411, -4.0174028172585015] MSE: 2.97164
_______________________________________________________________
Epoch: 479, Coefs:  [1.9593259975464072, -2.7478626047499994, -4.022434464989908] MSE: 2.97324
_______________________________________________________________
Epoch: 480, Coefs:  [1.9169908029502525, -2.7710

Epoch: 577, Coefs:  [1.6508918136402948, -2.092208841178664, -3.2308010716499225] MSE: 2.6444
_______________________________________________________________
Epoch: 578, Coefs:  [1.6495881411574247, -2.062830035941379, -3.200963222126006] MSE: 2.64902
_______________________________________________________________
Epoch: 579, Coefs:  [1.6529161260751166, -2.053090669983715, -3.188534749658294] MSE: 2.65441
_______________________________________________________________
Epoch: 580, Coefs:  [1.5795850709669934, -2.0968812617151853, -3.215478711742497] MSE: 2.58576
_______________________________________________________________
Epoch: 581, Coefs:  [1.515188385226438, -2.126957478926505, -3.230253698519685] MSE: 2.54283
_______________________________________________________________
Epoch: 582, Coefs:  [1.5091120426985063, -2.1278921442082717, -3.2304033057276462] MSE: 2.53942
_______________________________________________________________
Epoch: 583, Coefs:  [1.4946527860533763, -2.134173

...

Epoch: 986, Coefs:  [0.6830688890925285, -1.2787263915290936, -2.0448371255582547] MSE: 2.16049
_______________________________________________________________
Epoch: 987, Coefs:  [0.7009347807039147, -1.2789266808346054, -2.049695334375624] MSE: 2.16919
_______________________________________________________________
Epoch: 988, Coefs:  [0.658463246129359, -1.2973107895447662, -2.0579662924656783] MSE: 2.14577
_______________________________________________________________
Epoch: 989, Coefs:  [0.6660057493934728, -1.2929596918060098, -2.055389582113151] MSE: 2.1497
_______________________________________________________________
Epoch: 990, Coefs:  [0.6519026133949151, -1.3051989467466087, -2.0630579675910683] MSE: 2.14194
_______________________________________________________________
Epoch: 991, Coefs:  [0.6323055267718907, -1.3163268863849158, -2.068302313672854] MSE: 2.13349
_______________________________________________________________
Epoch: 992, Coefs:  [0.5791828733803408, -1.3

Epoch: 1152, Coefs:  [0.6265043264112038, -1.0135551046394518, -1.6922955341483146] MSE: 2.25426
_______________________________________________________________
Epoch: 1153, Coefs:  [0.5717502142907284, -1.0356940699933366, -1.7059182956251988] MSE: 2.19938
_______________________________________________________________
Epoch: 1154, Coefs:  [0.6339101217492544, -1.0007132515060255, -1.682310963667177] MSE: 2.26879
_______________________________________________________________
Epoch: 1155, Coefs:  [0.5684437050019474, -1.024112860625687, -1.6930115013742313] MSE: 2.20356
_______________________________________________________________
Epoch: 1156, Coefs:  [0.5640487905869221, -1.0216868136952335, -1.689535646011476] MSE: 2.20188
_______________________________________________________________
Epoch: 1157, Coefs:  [0.5743272339500376, -1.0204663839694292, -1.6925412635763217] MSE: 2.20937
_______________________________________________________________
Epoch: 1158, Coefs:  [0.5057716273270

Epoch: 1327, Coefs:  [0.12013178474042356, -1.0954958699006236, -1.5715426140354267] MSE: 2.0715
_______________________________________________________________
Epoch: 1328, Coefs:  [0.09163331683868045, -1.1020595936185331, -1.5733574195284863] MSE: 2.07783
_______________________________________________________________
Epoch: 1329, Coefs:  [0.11095668676219075, -1.0912086559824703, -1.5670943799877455] MSE: 2.0723
_______________________________________________________________
Epoch: 1330, Coefs:  [0.12596081915661023, -1.0888872437601558, -1.5673680370080687] MSE: 2.06979
_______________________________________________________________
Epoch: 1331, Coefs:  [0.07707388331458861, -1.103633664376618, -1.5725575517762842] MSE: 2.08144
_______________________________________________________________
Epoch: 1332, Coefs:  [0.05020718357294564, -1.1203935133009784, -1.5835498570718187] MSE: 2.09328
_______________________________________________________________
Epoch: 1333, Coefs:  [0.0717437

Epoch: 1468, Coefs:  [0.053557709807727935, -1.0422050952669542, -1.4423219652272683] MSE: 2.06902
_______________________________________________________________
Epoch: 1469, Coefs:  [0.13067587776934836, -0.9903637845542811, -1.4057560339132773] MSE: 2.06442
_______________________________________________________________
Epoch: 1470, Coefs:  [0.18786595865893949, -0.958072892923024, -1.3818241069730393] MSE: 2.07733
_______________________________________________________________
Epoch: 1471, Coefs:  [0.1798843741145961, -0.9664564547888835, -1.3862225414797076] MSE: 2.07414
_______________________________________________________________
Epoch: 1472, Coefs:  [0.1884982379556314, -0.943309063578117, -1.3642648962523058] MSE: 2.0811
_______________________________________________________________
Epoch: 1473, Coefs:  [0.19698049617421584, -0.9433871948876988, -1.3654685604639374] MSE: 2.08324
_______________________________________________________________
Epoch: 1474, Coefs:  [0.14265372

([-0.05079589511922756, -1.1667819751875135, -1.521352966350741],
 2.137790857889051,
 4.419507755297133)

# (D)

## experiment(N, d, var, M, details, weightDecay):

In [108]:
def experiment(N, d, var, weightDecay, M=50, details=False):
    
    training_data_df = getData(N,var)
    
    polynomials = []
    E_ins = []
    E_outs = []
    
    if details: print("_____________________________________________________________________ START TRIALS\n")
    
    for i in range(M):
        if details: print("______________________________________ M="+str(i+1))
        
        (p,ein,eout) = fitData(training_data_df, d, var, weightDecay, learning_rate=0.01, epochs=1000, testing_size=2000, details=False)
        
        polynomials.append(p)
        E_ins.append(ein)
        E_outs.append(eout)
        
        if details: print("FINAL MODEL: Coefs: ", p, " E_in: "+ str(ein)+" E_out: "+str(eout))
        
    if details: print("\n_____________________________________________________________________ AVERAGE POLYNOMIALS\n")
    
    E_in_bar = statistics.mean(E_ins)
    E_out_bar = statistics.mean(E_outs)
    
    average_polynomial = []
    transpose_coefs = transpose(polynomials)
    
    for i in range(d+1):
        average_polynomial.append(statistics.mean(transpose_coefs[i]))
        
    if details: print("AVERAGE MODEL: Coefs: ", average_polynomial, " E_in_bar: "+ str(round(E_in_bar,5))+" E_out_bar: "+str(round(E_out_bar,5)))
        
        
    if details: print("\n_____________________________________________________________________ TESTING\n")
        
    testing_data_df = getData(2000, var)
    
    E_bias = getMSE(testing_data_df, average_polynomial)
    
    if details: print("E_bias: "+str(E_bias))
    
    return (E_in_bar, E_out_bar, E_bias)

In [109]:
#Check

experiment(N=200, d=2, var=0.01, M=5, weightDecay=False, details=True)

_____________________________________________________________________ START TRIALS

______________________________________ M=1
FINAL MODEL: Coefs:  [0.7295492274748114, -4.342957769487267, 4.295513760093165]  E_in: 0.16270930729550076 E_out: 0.17932282223956786
______________________________________ M=2
FINAL MODEL: Coefs:  [0.2397551809541844, -1.1014232063705411, 1.209648360037148]  E_in: 0.38269328106465295 E_out: 0.4196950689849396
______________________________________ M=3
FINAL MODEL: Coefs:  [0.8077961121425713, -5.140597648558007, 5.1517882516025875]  E_in: 0.12730770085851814 E_out: 0.1369176252163551
______________________________________ M=4
FINAL MODEL: Coefs:  [0.7425034817105411, -4.201869317081889, 4.325671611688863]  E_in: 0.16964767300438488 E_out: 0.17976539938459257
______________________________________ M=5
FINAL MODEL: Coefs:  [1.2536103011467776, -7.349220361577732, 7.314550953632972]  E_in: 0.06041077747273199 E_out: 0.06828289336516181

_________________________

(0.18055374793915774, 0.1967967618381234, 0.17076534419757428)

# (E) Mini-Batch Stochastic Gradient Descent (batch_size=1)

In [110]:
Nes = [2,5,10,20,50,100,200]
Des = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
Vares = [0.01, 0.1, 1]

In [111]:
nn = []
dd = []
varvar = []
einbars = []
eoutbars = []
ebiass = []

for myN in Nes:
    for myD in Des:
        for myVar in Vares:
            print("_________________________________________________________________________")
            print("N = "+str(myN)+"; d = "+str(myD)+"; var = "+str(myVar))
            [E_in_bar, E_out_bar, E_bias] = experiment(N=myN, d=myD, var=myVar, M=50, weightDecay=False, details=False)
            print("E_in_bar = "+str(E_in_bar)+"; E_out_bar = "+str(E_out_bar)+"; E_bias = "+str(E_bias))
            
            nn.append(myN)
            dd.append(myD)
            varvar.append(myVar)
            einbars.append(E_in_bar)
            eoutbars.append(E_out_bar)
            ebiass.append(E_bias)
            
output_df = pd.DataFrame(list(zip(nn,dd,varvar,einbars,eoutbars,ebiass)), columns =['N','d','var','E_in_bar','E_out_bar','E_bias']) 

_________________________________________________________________________
N = 2; d = 0; var = 0.01
E_in_bar = 0.07062719158021848; E_out_bar = 0.6138300545736125; E_bias = 0.6084485623125929
_________________________________________________________________________
N = 2; d = 0; var = 0.1
E_in_bar = 0.9774501452273001; E_out_bar = 0.6079003665785762; E_bias = 0.5948483547736598
_________________________________________________________________________
N = 2; d = 0; var = 1
E_in_bar = 0.4262397212222037; E_out_bar = 1.7515851296656502; E_bias = 1.7496809755686564
_________________________________________________________________________
N = 2; d = 1; var = 0.01
E_in_bar = 0.005385646674906809; E_out_bar = 5.91714078338758; E_bias = 0.5740453649738998
_________________________________________________________________________
N = 2; d = 1; var = 0.1
E_in_bar = 0.010047755460305218; E_out_bar = 7.508977290171443; E_bias = 0.651318167109839
______________________________________________________

E_in_bar = 0.01928382968435205; E_out_bar = 32.26364158818723; E_bias = 1.5342767957134484
_________________________________________________________________________
N = 2; d = 14; var = 1
E_in_bar = 7.239112718145386e-05; E_out_bar = 5.066786338527338; E_bias = 1.5353855076772631
_________________________________________________________________________
N = 2; d = 15; var = 0.01
E_in_bar = 0.09107522800982588; E_out_bar = 21.980533095418775; E_bias = 4.308863470670258
_________________________________________________________________________
N = 2; d = 15; var = 0.1
E_in_bar = 0.3177836536077998; E_out_bar = 44.05538042321602; E_bias = 3.7146634083647103
_________________________________________________________________________
N = 2; d = 15; var = 1
E_in_bar = 0.511313717975119; E_out_bar = 42.837069211026474; E_bias = 6.143058547533603
_________________________________________________________________________
N = 2; d = 16; var = 0.01
E_in_bar = 0.40646259520585576; E_out_bar = 25.672066

E_in_bar = 0.3251915160929719; E_out_bar = 1.2964045689398997; E_bias = 0.38262584473889805
_________________________________________________________________________
N = 5; d = 8; var = 0.1
E_in_bar = 0.4005618584075655; E_out_bar = 0.8514191098884539; E_bias = 0.510445761244659
_________________________________________________________________________
N = 5; d = 8; var = 1
E_in_bar = 0.5084438962495496; E_out_bar = 5.300587493574133; E_bias = 2.586823959479363
_________________________________________________________________________
N = 5; d = 9; var = 0.01
E_in_bar = 0.2733063500076694; E_out_bar = 1.2618534955959528; E_bias = 0.28237141344803524
_________________________________________________________________________
N = 5; d = 9; var = 0.1
E_in_bar = 0.3309933330493319; E_out_bar = 1.4724846701573076; E_bias = 0.598187215805108
_________________________________________________________________________
N = 5; d = 9; var = 1
E_in_bar = 0.27735922139709807; E_out_bar = 8.12951796158184

E_in_bar = 0.3915124465868352; E_out_bar = 1.572922896113601; E_bias = 1.587238709292008
_________________________________________________________________________
N = 10; d = 2; var = 0.01
E_in_bar = 0.5018950123635991; E_out_bar = 0.5762845691967485; E_bias = 0.4670216152726549
_________________________________________________________________________
N = 10; d = 2; var = 0.1
E_in_bar = 0.46843903603148196; E_out_bar = 0.5314543311597334; E_bias = 0.4486099576734642
_________________________________________________________________________
N = 10; d = 2; var = 1
E_in_bar = 0.5534654207684865; E_out_bar = 1.5230125282135316; E_bias = 1.4150184856372068
_________________________________________________________________________
N = 10; d = 3; var = 0.01
E_in_bar = 0.3939263579356903; E_out_bar = 0.5188295767507988; E_bias = 0.32149587940838775
_________________________________________________________________________
N = 10; d = 3; var = 0.1
E_in_bar = 0.3523518154368889; E_out_bar = 2.66729

E_in_bar = 0.38957897167265854; E_out_bar = 2.875328968027082; E_bias = 0.41091728954403856
_________________________________________________________________________
N = 10; d = 16; var = 1
E_in_bar = 0.9971828001866455; E_out_bar = 6.027200100751887; E_bias = 1.533326933421326
_________________________________________________________________________
N = 10; d = 17; var = 0.01
E_in_bar = 0.21530083176120793; E_out_bar = 1.4375523792988598; E_bias = 0.4255137178158259
_________________________________________________________________________
N = 10; d = 17; var = 0.1
E_in_bar = 0.4691581205346783; E_out_bar = 0.6563183557510124; E_bias = 0.34121699902016506
_________________________________________________________________________
N = 10; d = 17; var = 1
E_in_bar = 0.7920724427736765; E_out_bar = 1.576549353880423; E_bias = 1.353673268956083
_________________________________________________________________________
N = 10; d = 18; var = 0.01
E_in_bar = 0.3560183251061198; E_out_bar = 0.682

...

E_in_bar = 0.3714102015491239; E_out_bar = 0.29898262046544216; E_bias = 0.22523997899988538
_________________________________________________________________________
N = 50; d = 3; var = 1
E_in_bar = 1.0678902423085876; E_out_bar = 1.450715793622681; E_bias = 1.3632430718508854
_________________________________________________________________________
N = 50; d = 4; var = 0.01
E_in_bar = 0.18063074743145388; E_out_bar = 0.24892358167617282; E_bias = 0.15819485806508765
_________________________________________________________________________
N = 50; d = 4; var = 0.1
E_in_bar = 0.24433469535411842; E_out_bar = 0.3908265237143457; E_bias = 0.31413305129482433
_________________________________________________________________________
N = 50; d = 4; var = 1
E_in_bar = 1.2432946801308702; E_out_bar = 1.1919828046209955; E_bias = 1.1691998730838895
_________________________________________________________________________
N = 50; d = 5; var = 0.01
E_in_bar = 0.18712893367722483; E_out_bar = 0.

E_in_bar = 0.9794024047405657; E_out_bar = 1.2437897441401857; E_bias = 1.0801114070709827
_________________________________________________________________________
N = 50; d = 18; var = 0.01
E_in_bar = 0.2682726175894386; E_out_bar = 0.25571131523488416; E_bias = 0.12524993170695817
_________________________________________________________________________
N = 50; d = 18; var = 0.1
E_in_bar = 0.27391242357530166; E_out_bar = 2.6054786669432497; E_bias = 0.7193774235647504
_________________________________________________________________________
N = 50; d = 18; var = 1
E_in_bar = 1.1432118739150585; E_out_bar = 1.2661497999434592; E_bias = 1.1215268907449205
_________________________________________________________________________
N = 50; d = 19; var = 0.01
E_in_bar = 0.21102165446989302; E_out_bar = 0.23404440847644084; E_bias = 0.1284408856460165
_________________________________________________________________________
N = 50; d = 19; var = 0.1
E_in_bar = 0.27285539586797847; E_out_ba

E_in_bar = 0.13410072281149457; E_out_bar = 0.14498004309689014; E_bias = 0.09655725798858208
_________________________________________________________________________
N = 100; d = 11; var = 0.1
E_in_bar = 0.19292747977329644; E_out_bar = 0.22235579962597568; E_bias = 0.1731449030799229
_________________________________________________________________________
N = 100; d = 11; var = 1
E_in_bar = 1.0742835062848586; E_out_bar = 1.1302523878644262; E_bias = 1.0769450296359018
_________________________________________________________________________
N = 100; d = 12; var = 0.01
E_in_bar = 0.10376014825380683; E_out_bar = 0.11185615288527259; E_bias = 0.06316086258989466
_________________________________________________________________________
N = 100; d = 12; var = 0.1
E_in_bar = 0.19964657353112422; E_out_bar = 0.19963425951249572; E_bias = 0.14703750787825834
_________________________________________________________________________
N = 100; d = 12; var = 1
E_in_bar = 1.1170894056739948; E

E_in_bar = 0.20209600229614913; E_out_bar = 0.19125842293752435; E_bias = 0.17809744944517933
_________________________________________________________________________
N = 200; d = 4; var = 1
E_in_bar = 1.1856347279469193; E_out_bar = 1.1306652769099426; E_bias = 1.0721470222991611
_________________________________________________________________________
N = 200; d = 5; var = 0.01
E_in_bar = 0.09051908299643559; E_out_bar = 0.0987197015150169; E_bias = 0.07803254875640775
_________________________________________________________________________
N = 200; d = 5; var = 0.1
E_in_bar = 0.1950626813084585; E_out_bar = 0.212710121091378; E_bias = 0.17916781230250628
_________________________________________________________________________
N = 200; d = 5; var = 1
E_in_bar = 1.1548537305097535; E_out_bar = 1.1749981654974548; E_bias = 1.1456503386752348
_________________________________________________________________________
N = 200; d = 6; var = 0.01
E_in_bar = 0.1333561384419184; E_out_bar =

E_in_bar = 1.1382194208593328; E_out_bar = 1.185071081519007; E_bias = 1.102607608483389
_________________________________________________________________________
N = 200; d = 19; var = 0.01
E_in_bar = 0.11240557019854297; E_out_bar = 0.11730738648558454; E_bias = 0.05096497622353829
_________________________________________________________________________
N = 200; d = 19; var = 0.1
E_in_bar = 0.22677628156649277; E_out_bar = 0.21483072040137519; E_bias = 0.16647250350225737
_________________________________________________________________________
N = 200; d = 19; var = 1
E_in_bar = 1.0297581271782865; E_out_bar = 1.1499204215715706; E_bias = 1.0804986197200432
_________________________________________________________________________
N = 200; d = 20; var = 0.01
E_in_bar = 0.11124695615618713; E_out_bar = 0.10649529256651588; E_bias = 0.06308176926360201
_________________________________________________________________________
N = 200; d = 20; var = 0.1
E_in_bar = 0.1908719442496382; E_

In [112]:
output_df.to_excel('output.xlsx') 

# (F) Mini-Batch Stochastic Gradient Descent with Weight Decay (batch_size=1)

In [113]:
def MiniBatchStochasticGradientDescent_WeightDecay(df, d, learning_rate, epochs, weight_decay=0.1, details=False):
    
    augmented_df = augmentedXDataframe(df, d)      #Add polynomial X terms to dataframe
    
    N = len(augmented_df['X'])
    
    # START FROM A RANDOM NON-ZERO THETA _____________________
    
    theta = []
    
    for i in range(d+1):
        num = 0
        while num==0:
            num = random.randint(-10,10)
        theta.append(num)
        
    if details: print("Random Theta: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
    
    # UPDATE THETA EPOCHS TIMES ______________________________
    
    for k in range(epochs):
        
        if details: print("_______________________________________________________________")
            
         # Choose random indexes in N for batch
        randomIndexes = []
        batch_size= 1             #Stochastic gradient descent to save time
        
        for i in range(batch_size):  
            randomIndexes.append(random.randint(0, N-1))
        
        for j in range(d+1):  # for each coefficient
            
            derivative = 0
            
            # GET DERIVATIVE for theta[j]
            for i in randomIndexes:
                derivative = (weight_decay * derivative) + 2 * (df['Y'][i] - getPolynomialY(theta, df['X'][i])) * augmented_df['X_'+str(j)][i]
            
            # UPDATE THETA
            theta[j] += learning_rate * derivative
        
        if details: print("Epoch: "+str(k)+", Coefs: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
        
    return theta
    
    
    return estim_poly_coefs

#MiniBatchStochasticGradientDescent_WeightDecay(test_df, 3, 0.01, epochs=1500, details=True)

In [114]:
nn = []
dd = []
varvar = []
einbars = []
eoutbars = []
ebiass = []

for myN in Nes:
    for myD in Des:
        for myVar in Vares:
            print("_________________________________________________________________________")
            print("N = "+str(myN)+"; d = "+str(myD)+"; var = "+str(myVar))
            [E_in_bar, E_out_bar, E_bias] = experiment(N=myN, d=myD, var=myVar, M=50, weightDecay=True, details=False)
            print("E_in_bar = "+str(E_in_bar)+"; E_out_bar = "+str(E_out_bar)+"; E_bias = "+str(E_bias))
            
            nn.append(myN)
            dd.append(myD)
            varvar.append(myVar)
            einbars.append(E_in_bar)
            eoutbars.append(E_out_bar)
            ebiass.append(E_bias)
            
output_df = pd.DataFrame(list(zip(nn,dd,varvar,einbars,eoutbars,ebiass)), columns =['N','d','var','E_in_bar','E_out_bar','E_bias']) 

_________________________________________________________________________
N = 2; d = 0; var = 0.01
E_in_bar = 0.2602867312992623; E_out_bar = 0.6113591776602709; E_bias = 0.6309153168014875
_________________________________________________________________________
N = 2; d = 0; var = 0.1
E_in_bar = 0.4539384118584217; E_out_bar = 0.6225796967538627; E_bias = 0.6129439124858085
_________________________________________________________________________
N = 2; d = 0; var = 1
E_in_bar = 1.632870322800514; E_out_bar = 1.5616326740926758; E_bias = 1.502709484785716
_________________________________________________________________________
N = 2; d = 1; var = 0.01
E_in_bar = 0.44043040561293867; E_out_bar = 2.1510545814905764; E_bias = 0.8773142290892506
_________________________________________________________________________
N = 2; d = 1; var = 0.1
E_in_bar = 0.41800668373816036; E_out_bar = 2.1547785111884905; E_bias = 1.38654711310346
_________________________________________________________

E_in_bar = 4.9309290439456775e-08; E_out_bar = 5.297560203831223; E_bias = 1.4938181416143939
_________________________________________________________________________
N = 2; d = 14; var = 1
E_in_bar = 0.8259910167931831; E_out_bar = 32.77527764660004; E_bias = 4.229987176412666
_________________________________________________________________________
N = 2; d = 15; var = 0.01
E_in_bar = 0.0889410383389861; E_out_bar = 26.66883973541815; E_bias = 1.6906071656164559
_________________________________________________________________________
N = 2; d = 15; var = 0.1
E_in_bar = 0.7331140108119081; E_out_bar = 34.7462587868418; E_bias = 6.717203671758638
_________________________________________________________________________
N = 2; d = 15; var = 1
E_in_bar = 0.00392857483585941; E_out_bar = 16.575929148272277; E_bias = 1.6192922331527293
_________________________________________________________________________
N = 2; d = 16; var = 0.01
E_in_bar = 0.2538840832157527; E_out_bar = 30.36679650

E_in_bar = 0.5130446097329399; E_out_bar = 2.977814097618381; E_bias = 0.2063571447655276
_________________________________________________________________________
N = 5; d = 8; var = 0.1
E_in_bar = 0.5463968214399145; E_out_bar = 2.1944950800096428; E_bias = 0.5115094947170874
_________________________________________________________________________
N = 5; d = 8; var = 1
E_in_bar = 0.7012393278862709; E_out_bar = 3.956148255570845; E_bias = 2.1077873980159145
_________________________________________________________________________
N = 5; d = 9; var = 0.01
E_in_bar = 0.7438821980036652; E_out_bar = 2.5344783633739714; E_bias = 0.43363247192027754
_________________________________________________________________________
N = 5; d = 9; var = 0.1
E_in_bar = 0.8677301216596657; E_out_bar = 1.1244312916947656; E_bias = 0.39319367265903044
_________________________________________________________________________
N = 5; d = 9; var = 1
E_in_bar = 0.5415710159217888; E_out_bar = 3.2604530994892

E_in_bar = 1.1451454189346584; E_out_bar = 1.8693390989215664; E_bias = 1.5961797709399916
_________________________________________________________________________
N = 10; d = 2; var = 0.01
E_in_bar = 0.32235702565378677; E_out_bar = 0.6692946514875137; E_bias = 0.5786140548854471
_________________________________________________________________________
N = 10; d = 2; var = 0.1
E_in_bar = 1.108564776225375; E_out_bar = 0.6380067676457222; E_bias = 0.4790026272586473
_________________________________________________________________________
N = 10; d = 2; var = 1
E_in_bar = 0.9423050092411237; E_out_bar = 1.7775137885422099; E_bias = 1.6327908727805651
_________________________________________________________________________
N = 10; d = 3; var = 0.01
E_in_bar = 0.4192608948643085; E_out_bar = 0.7044591507350073; E_bias = 0.47525707116999205
_________________________________________________________________________
N = 10; d = 3; var = 0.1
E_in_bar = 0.43265001823193916; E_out_bar = 0.848

E_in_bar = 0.6125323933865919; E_out_bar = 1.0021362364562083; E_bias = 0.4999138862888838
_________________________________________________________________________
N = 10; d = 16; var = 1
E_in_bar = 0.8229165170437884; E_out_bar = 2.6828123875782333; E_bias = 1.5391151626224413
_________________________________________________________________________
N = 10; d = 17; var = 0.01
E_in_bar = 0.502684044973994; E_out_bar = 9.739904152968895; E_bias = 0.39213354965003155
_________________________________________________________________________
N = 10; d = 17; var = 0.1
E_in_bar = 0.6123778974487376; E_out_bar = 1.551964826801556; E_bias = 0.4641468067633336
_________________________________________________________________________
N = 10; d = 17; var = 1
E_in_bar = 1.4217350446024073; E_out_bar = 3.7967096977839585; E_bias = 2.5570440361502653
_________________________________________________________________________
N = 10; d = 18; var = 0.01
E_in_bar = 0.3429133217450164; E_out_bar = 0.5381

...

E_in_bar = 1.2124578868718436; E_out_bar = 1.6971944585561882; E_bias = 1.531848693290626
_________________________________________________________________________
N = 50; d = 4; var = 0.01
E_in_bar = 0.620972560373614; E_out_bar = 0.7587532738110551; E_bias = 0.42045536825761454
_________________________________________________________________________
N = 50; d = 4; var = 0.1
E_in_bar = 0.8470844767627402; E_out_bar = 0.8302849473196776; E_bias = 0.46245109959733943
_________________________________________________________________________
N = 50; d = 4; var = 1
E_in_bar = 1.5794370860019828; E_out_bar = 1.8380814699963854; E_bias = 1.4647499713852885
_________________________________________________________________________
N = 50; d = 5; var = 0.01
E_in_bar = 0.6410991149380012; E_out_bar = 0.5972058106251956; E_bias = 0.2998577534741474
_________________________________________________________________________
N = 50; d = 5; var = 0.1
E_in_bar = 0.8448564747537921; E_out_bar = 1.00124

E_in_bar = 0.788196882894001; E_out_bar = 0.8543842042236957; E_bias = 0.2956279564764768
_________________________________________________________________________
N = 50; d = 18; var = 1
E_in_bar = 2.06090535919166; E_out_bar = 2.002814605949242; E_bias = 1.3468638787859877
_________________________________________________________________________
N = 50; d = 19; var = 0.01
E_in_bar = 0.7250387742619789; E_out_bar = 0.6724091025412858; E_bias = 0.33180343720115046
_________________________________________________________________________
N = 50; d = 19; var = 0.1
E_in_bar = 1.048173618895988; E_out_bar = 1.7695860419650735; E_bias = 0.5689737405662925
_________________________________________________________________________
N = 50; d = 19; var = 1
E_in_bar = 1.9027960744927515; E_out_bar = 2.0106913627430028; E_bias = 1.4434464494817638
_________________________________________________________________________
N = 50; d = 20; var = 0.01
E_in_bar = 0.7681002222054114; E_out_bar = 0.909089

E_in_bar = 1.6933574001706166; E_out_bar = 1.6643461751418567; E_bias = 1.3117442112125886
_________________________________________________________________________
N = 100; d = 12; var = 0.01
E_in_bar = 0.6201377964788806; E_out_bar = 0.6196340732502824; E_bias = 0.286104166335936
_________________________________________________________________________
N = 100; d = 12; var = 0.1
E_in_bar = 0.8750162636618805; E_out_bar = 0.8946754400006643; E_bias = 0.43785812819231495
_________________________________________________________________________
N = 100; d = 12; var = 1
E_in_bar = 1.8240491125970157; E_out_bar = 2.479754515371242; E_bias = 1.1967382577694652
_________________________________________________________________________
N = 100; d = 13; var = 0.01
E_in_bar = 0.842392203231292; E_out_bar = 1.0988572904462575; E_bias = 0.29329854798414506
_________________________________________________________________________
N = 100; d = 13; var = 0.1
E_in_bar = 0.8218467288119908; E_out_bar 

E_in_bar = 0.6129571691343784; E_out_bar = 0.5727538546985724; E_bias = 0.2381127169362547
_________________________________________________________________________
N = 200; d = 5; var = 0.1
E_in_bar = 0.8420437261268593; E_out_bar = 0.8488252416905965; E_bias = 0.4144808089788856
_________________________________________________________________________
N = 200; d = 5; var = 1
E_in_bar = 1.6973303642034896; E_out_bar = 1.7186490787868107; E_bias = 1.3452312508792037
_________________________________________________________________________
N = 200; d = 6; var = 0.01
E_in_bar = 0.7856014483468202; E_out_bar = 0.8389245631120636; E_bias = 0.2557766384368774
_________________________________________________________________________
N = 200; d = 6; var = 0.1
E_in_bar = 0.8471582589083858; E_out_bar = 0.837831998214479; E_bias = 0.4663725173748154
_________________________________________________________________________
N = 200; d = 6; var = 1
E_in_bar = 1.839799846678276; E_out_bar = 1.75631

E_in_bar = 0.8903430214230981; E_out_bar = 0.8509649918578894; E_bias = 0.28848525680860354
_________________________________________________________________________
N = 200; d = 19; var = 1
E_in_bar = 1.5176064420000204; E_out_bar = 2.0592615041234037; E_bias = 1.4603028673399308
_________________________________________________________________________
N = 200; d = 20; var = 0.01
E_in_bar = 0.8104478642384291; E_out_bar = 0.8690394798114732; E_bias = 0.2806047090035212
_________________________________________________________________________
N = 200; d = 20; var = 0.1
E_in_bar = 0.9173264192322046; E_out_bar = 0.9692465027686598; E_bias = 0.39444011586000277
_________________________________________________________________________
N = 200; d = 20; var = 1
E_in_bar = 1.8373162806544518; E_out_bar = 1.946049589756396; E_bias = 1.3959016719657469


In [115]:
output_df.to_excel('output_weight_decay.xlsx') 